In [1]:
! pip install pandas
! pip install petl
! pip install glob


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import petl as etl
import os
import glob

1. Creating dataframe of all JSON files

In [3]:
# JSON FILES APPENDING TO A SINGLE DATAFRAME
path = "branch_data/sales_by_branch"

all_json_files = glob.glob(os.path.join(path, "*.json"))

df_from_each_json_file = (pd
                    .read_json(file)
                    .rename(columns={
                        'total_quantity': 'quantity', 
                        'quantity_purchased': 'quantity',
                        'total_quantity_purchased': 'quantity',
                        'item': 'product',
                        'sku': 'product'
                        })
                    .assign(branch = file[38:][:-5].replace('_', ' ')) 
                    for file in all_json_files )

df_json_merged = pd.concat(df_from_each_json_file, ignore_index=True)
df_json_merged.head()

,year,month,day,hour,product,quantity,amount_in_gbp,branch
0,2010,1,1,0,Sweet potatoes loose,160,140.800,Ceredigion store
1,2010,1,1,0,Plain Greek yoghurt,59,47.200,Ceredigion store
2,2010,1,1,0,sea bass fillets,163,365.120,Ceredigion store
3,2010,1,1,0,grey cardigan,222,3552.000,Ceredigion store
4,2010,1,1,0,british parsnips loose,144,96.768,Ceredigion store


In [4]:
# renaming dataframe to be reused for both CSV and JSON dataframes
sales_df = df_json_merged

2. Formatting of main dataframes

In [5]:
'''1 Convert to table'''
sales_df = etl.fromdataframe(sales_df)
print(sales_df)

+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| year | month | day | hour | product                | quantity | amount_in_gbp      | branch           |
+======+=======+=====+======+========================+==========+====================+==================+
| 2010 |     1 |   1 |    0 | Sweet potatoes loose   |      160 |              140.8 | Ceredigion store |
+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| 2010 |     1 |   1 |    0 | Plain Greek yoghurt    |       59 |               47.2 | Ceredigion store |
+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| 2010 |     1 |   1 |    0 | sea bass fillets       |      163 | 365.11999999999995 | Ceredigion store |
+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| 2010 |     1 |   1 |    0 | grey cardigan   

In [6]:
'''2 replace '-' with '0' '''
sales_df = etl.replaceall(sales_df, '-', '0')
print(sales_df)

+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| year | month | day | hour | product                | quantity | amount_in_gbp      | branch           |
+======+=======+=====+======+========================+==========+====================+==================+
| 2010 |     1 |   1 |    0 | Sweet potatoes loose   |      160 |              140.8 | Ceredigion store |
+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| 2010 |     1 |   1 |    0 | Plain Greek yoghurt    |       59 |               47.2 | Ceredigion store |
+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| 2010 |     1 |   1 |    0 | sea bass fillets       |      163 | 365.11999999999995 | Ceredigion store |
+------+-------+-----+------+------------------------+----------+--------------------+------------------+
| 2010 |     1 |   1 |    0 | grey cardigan   

In [7]:
'''4 convert to numbers'''
sales_df = etl.addfield(sales_df, 'qty', lambda row: int(row.quantity))
sales_df = etl.cutout(sales_df, 'quantity')
sales_df



year,month,day,hour,product,amount_in_gbp,branch,qty
2010,1,1,0,Sweet potatoes loose,140.8,Ceredigion store,160
2010,1,1,0,Plain Greek yoghurt,47.2,Ceredigion store,59
2010,1,1,0,sea bass fillets,365.11999999999995,Ceredigion store,163
2010,1,1,0,grey cardigan,3552.0,Ceredigion store,222
2010,1,1,0,british parsnips loose,96.768,Ceredigion store,144


3. Save table to CSV file
    - this is quicker than converting to dataframe

In [8]:
etl.tocsv(sales_df,'data_output/1_json_files_dataframe.csv')